# Predictive maintenance of Lathe machine

#### Importing important libraries and modules

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from scipy import signal
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

In [2]:
import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### Importing Vibration Data and converting it to proper Time-series format.

In [3]:
#Experiment details
exp = pd.read_excel("Data/Experiment Summary.xlsx") 

In [4]:
dataframes = []  # To store the imported data from each file

for i in range(1, 61):
    file = f"Data/{i}.xlsx"
    df = pd.read_excel(file)  # Use pd.read_excel() for Excel files
    df = df.dropna(axis='columns', how='all')  
    df = df.dropna(axis='rows', how='all') 
    df.columns = ['Time', 'X', 'Y', 'Z']
    df = df.iloc[1:]  # Exclude the original header row from the data
    df['Time'] = pd.to_datetime(df['Time'], unit='s').dt.time  # Convert 'Time' column to datetime
    
    # Add experiment details from exp dataframe based on experiment number
    experiment_number = i  # Or any other way to determine the experiment number
    experiment_row = exp[exp['Experiment'] == experiment_number]
    if not experiment_row.empty:
        for column in experiment_row.columns[1:]:
            df[column] = experiment_row[column].values[0]
    
    dataframes.append(df)
    
    print(i)  # To keep an eye on progress


for i in range(len(dataframes)):
    dataframes[i]['X'] = pd.to_numeric(dataframes[i]['X'], errors='coerce')
    dataframes[i]['Y'] = pd.to_numeric(dataframes[i]['Y'], errors='coerce')
    dataframes[i]['Z'] = pd.to_numeric(dataframes[i]['Z'], errors='coerce')
    dataframes[i]['Magnitude'] = np.sqrt(dataframes[i]['X']**2 + dataframes[i]['Y']**2 + dataframes[i]['Z']**2)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60


In [5]:
merged_df = pd.concat(dataframes, ignore_index=True)
merged_df['Time'] = [t.hour * 3600 + t.minute * 60 + t.second + t.microsecond / 1e6 for t in merged_df['Time']]


## Exploratory Data Analyis

In [ ]:

# Compute the correlation matrix
corr=merged_df.corr(numeric_only=True)
plt.figure(figsize=(8, 8))
sns.heatmap(corr, square=True,annot=True,cmap='YlGnBu',linecolor="black")
plt.title('Correlation between features')
plt.show()


#### Following are the observations from correlation heat map:
    1) X_acceleration is the most prominent in the magnitude calculated.
    2) Surface roughness and feed rate are positively correlated 
    3) Surface roughness and RPM of spindle are negatively correlated.

#### Frequency Analysis

In [ ]:
# Calculating PSD

def calculate_psd(dataframe, fs=1000, w=10):
    time = dataframe['Time']
    mag = dataframe['Magnitude']
    x = dataframe['X']
    y = dataframe['Y']
    z = dataframe['Z']
    
    # Convert time values to seconds
    time_seconds = [(t.hour * 3600 + t.minute * 60 + t.second + t.microsecond / 1e6) for t in time]

    # Apply Hanning window function
    window = np.hanning(len(time_seconds))
    x_windowed = x * window
    y_windowed = y * window
    z_windowed = z * window
    mag_windowed = mag * window
    
    # Calculate PSD using periodogram
    f, psd_mag = signal.welch(mag_windowed, fs, nperseg=len(time_seconds)/w)
    _, psd_x = signal.welch(x_windowed, fs, nperseg=len(time_seconds)/w)
    _, psd_y = signal.welch(y_windowed, fs, nperseg=len(time_seconds)/w)
    _, psd_z = signal.welch(z_windowed, fs, nperseg=len(time_seconds)/w)

    return f, psd_mag, psd_x, psd_y, psd_z


# Time_domain plot
def time_domain(num):
    df = dataframes[num]

    fig = px.line(df, x='Time', y='Magnitude', title='Vibration Sensor Data', labels={'Magnitude': 'Acceleration'})

    fig.update_layout(
        height=600,
        showlegend=True,
        paper_bgcolor='darkgrey'
    )
    fig.show()
    


#Plotting PSD
def freq_domain(frequencies, psd_mag, psd_x, psd_y, psd_z):
    # Convert complex PSD values to magnitude
    psd_mag_mag = np.abs(psd_mag)
    psd_x_mag = np.abs(psd_x)
    psd_y_mag = np.abs(psd_y)
    psd_z_mag = np.abs(psd_z)
    # Create line plots for X, Y, and Z axes
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=frequencies, y=psd_mag_mag, mode='lines', name='Magnitude'))
    fig.add_trace(go.Scatter(x=frequencies, y=psd_x_mag, mode='lines', name='X'))
    fig.add_trace(go.Scatter(x=frequencies, y=psd_y_mag, mode='lines', name='Y'))
    fig.add_trace(go.Scatter(x=frequencies, y=psd_z_mag, mode='lines', name='Z'))
    fig.update_layout(
        title='Power Spectral Density',
        xaxis=dict(title='Frequency'),
        yaxis=dict(title='acceleration', type='log'),
        showlegend=True,
        paper_bgcolor= 'darkgrey',
        height=600


    )

    fig.show()




In [ ]:
# Calling the function with the DataFrames
for j in range(5):
    for i in range(j, 60, 20):
        print(i)
        print(exp.iloc[i])
        new = time_domain(i) 
        experiment, mag, x, y, z = calculate_psd(dataframes[i], fs=1000)
        plo = freq_domain(experiment, mag, x, y, z)
        
        print()
    

In [ ]:
# Compute the spectrogram using STFT
frequencies, times, spectrogram = signal.spectrogram(merged_df['Magnitude'], fs=1000)

# Plot the time-frequency representation
plt.pcolormesh(times, frequencies, 10 * np.log10(spectrogram), shading='auto', cmap='inferno')

# Configure the plot
plt.xlabel('Time')
plt.ylabel('Frequency')
plt.title('Magnitude Time-Frequency Plot')
plt.colorbar(label='Power Spectral Density (dB)')

# Show the plot
plt.show()

## Model

#### Data split

In [6]:

# Step 1: Prepare the Data
# Split the data into features (X) and target variable (y)
X = merged_df.drop('Ra', axis=1)
y = merged_df['Ra']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)


In [7]:
from sklearn.utils import shuffle

# Shuffle the test data sets correspondingly
X_train, y_train = shuffle(X_train, y_train, random_state=42)


### Random Forest

In [ ]:
# USING CPU COMPUTATION

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# 1. Data Preparation - Assume X and y contain the features and target variable

# 2. Import Libraries

# 4. Create Random Forest Model
model = RandomForestRegressor(n_estimators=100, max_depth=10)

# 5. Fit the Model
model.fit(X_train, y_train)

# 6. Make Predictions
y_pred = model.predict(X_test)

# 7. Evaluate the Model
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)


In [ ]:
# USING GPU COMPUTATION

from sklearn.metrics import mean_absolute_error
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# 3. Create XGBoost Random Forest Model
model = XGBRegressor(n_estimators=300, tree_method='gpu_hist')

# 4. Fit the Model
model.fit(X_train, y_train)

# 5. Make Predictions
y_pred = model.predict(X_test)

# 6. Evaluate the Model
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

# Calculate the MAE
mae = mean_absolute_error(y_test, y_pred)
# Print the MAE
print('Mean Absolute Error:', mae)


In [ ]:
model.save_model('xgboost_model.bin')

In [ ]:
# Create a DataFrame with 'y_pred' and 'y_train' columns
comparison_df = pd.DataFrame({'y_pred': y_pred, 'y_test': y_test})

# Print the DataFrame
comparison_df.head(30)



### SVM

In [ ]:
import cupy as cp
from cuml import SVR

# 1. Transfer data to GPU
X_train_gpu = cp.asarray(X_train)
y_train_gpu = cp.asarray(y_train)
X_test_gpu = cp.asarray(X_test)

# 2. Create the GPU-based SVM model
model_gpu = SVR(kernel='rbf', C=1.0, epsilon=0.1)

# 3. Fit the model
model_gpu.fit(X_train_gpu, y_train_gpu)

# 4. Make predictions
y_pred_gpu = model_gpu.predict(X_test_gpu)

# 5. Transfer predictions back to CPU
y_pred_cpu = cp.asnumpy(y_pred_gpu)

# 6. Evaluate the model
mse = mean_squared_error(y_test, y_pred_cpu)
print('Mean Squared Error:', mse)

mae = mean_absolute_error(y_test, y_pred_cpu)
print('Mean Absolute Error:', mae)


### CNN

In [8]:
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
import keras_tuner as kt

tf.keras.mixed_precision.set_global_policy('mixed_float16')

# Scale the features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the model-building function
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Conv1D(hp.Int('conv_units', min_value=64, max_value=2048, step=64),
                                  kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)))
    model.add(keras.layers.MaxPooling1D(pool_size=2))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(hp.Int('dense_units', min_value=256, max_value=3200, step=128),
                                 activation='relu'))
    model.add(keras.layers.Dense(960, activation='linear'))

    optimizer = Adam(learning_rate=hp.Choice('learning_rate', values=[1e-5, 1e-6, 1e-7]))
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])
    return model

# Define the tuner
tuner = kt.Hyperband(build_model, objective='val_loss', max_epochs=10, factor=3, directory='tuner_results')

# Perform hyperparameter tuning
tuner.search(np.expand_dims(X_train_scaled, axis=-1), y_train, epochs=8, batch_size=5760, validation_data=(np.expand_dims(X_test_scaled, axis=-1), y_test))

# Get the best model
best_model = tuner.get_best_models(1)[0]

# Evaluate the best model
loss, mae = best_model.evaluate(np.expand_dims(X_test_scaled, axis=-1), y_test)
print('Mean Squared Error:', loss)
print('Mean Absolute Error:', mae)

# Make predictions
predictions = best_model.predict(np.expand_dims(X_test_scaled, axis=-1))


Trial 26 Complete [00h 00m 41s]
val_loss: 2.300213098526001

Best val_loss So Far: 2.097658395767212
Total elapsed time: 00h 08m 03s

Search: Running Trial #27

Hyperparameter    |Value             |Best Value So Far 
conv_units        |768               |512               
dense_units       |512               |1024              
learning_rate     |0.0001            |0.0001            
tuner/epochs      |10                |10                
tuner/initial_e...|0                 |4                 
tuner/bracket     |0                 |2                 
tuner/round       |0                 |2                 

Epoch 1/10
272/272 [==============================] - 5s 14ms/step - loss: 7.9534 - mae: 2.2425 - val_loss: 2.0401 - val_mae: 1.1012
Epoch 2/10
146/272 [===============>..............] - ETA: 1s - loss: 1.6799 - mae: 1.1459

KeyboardInterrupt: 

In [ ]:
from tensorflow.keras.optimizers import Adam

tf.keras.mixed_precision.set_global_policy('mixed_float16')

#Scale the features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#the CNN Model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv1D(512, kernel_size=4, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(, activation='relu'))
model.add(tf.keras.layers.Dense(1024, activation='linear'))

optimizer = Adam(learning_rate=0.00002)
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])

# Train the CNN Model
model.fit(np.expand_dims(X_train_scaled, axis=-1), y_train, epochs=8, batch_size=5760)

# Evaluate the CNN Model
loss, mae = model.evaluate(np.expand_dims(X_test_scaled, axis=-1), y_test)
print('Mean Squared Error:', loss)
print('Mean Absolute Error:', mae)

# Make predictions
predictions = model.predict(np.expand_dims(X_test_scaled, axis=-1))


In [ ]:
from tensorflow.keras.models import save_model

# Save the model
model.save('cnn.h5')


### LSTM

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM, Dense

tf.keras.mixed_precision.set_global_policy('mixed_float16')

# Scale the features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the LSTM Model
model = Sequential()
model.add(LSTM(256, activation='relu', input_shape=(X_train_scaled.shape[1], 1)))
model.add(Dense(512, activation='relu'))
model.add(Dense(64, activation='relu'))

optimizer = Adam(learning_rate=0.00001)
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])

# Train the LSTM Model
model.fit(np.expand_dims(X_train_scaled, axis=-1), y_train, epochs=10, batch_size=5760)

# Evaluate the LSTM Model
loss, mae = model.evaluate(np.expand_dims(X_test_scaled, axis=-1), y_test)
print('Mean Squared Error:', loss)
print('Mean Absolute Error:', mae)

# Make predictions
predictions = model.predict(np.expand_dims(X_test_scaled, axis=-1))
